In [2]:
import matplotlib
#matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os

#  SVM module
from PredictSVM import PredictMultiState
from PredictSVM import get_sessions, load_trial_times_whole_stack

from PredictSVM import PredictSVMTime

# 
data_dir = '/media/cat/4TBSSD/yuki/'
#data_dir = '/mnt/f418adc0-4ff9-410c-93c9-d6d21a1e9c35/yuki/'
#data_dir = '/media/cat/1TB/yuki/testdata/'

labels = ['left_paw', 
            'right_paw',
            'nose',
            'jaw',
            'right_ear',
            'tongue',
            'lever',
            'all',
            'code_04',
            'code_04_lockout']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


In [4]:
######################################################
##### PREDICT SVM DECISION CHOICE SINGLE SESSION #####
######################################################

# select animal names
names = ['IA1','IA2','IA3','IJ1','IJ2','AQ2'] # "AR4" and other datasets could work
names = ['IJ2']

# 
svm = PredictMultiState()
svm.root_dir = data_dir
svm.parallel = True
svm.n_cores = 30

# 
svm.no_movement = 3            # number of sec intra-feature lockout
svm.window = 15                # number of seconds pre movement to compute classification
svm.sliding_window = 30         # number of time steps that go into classifier
svm.sliding_window_step = 1    # number of frames to step the sliding window across
svm.code = 'multi_class'
svm.xvalidation = 10
svm.all_comps = 30

#
svm.animal_id = names[0]
svm.session_id = 'Feb1_'


#
svm.classifier = 'svm'               # 'svm',  'random_forest'
svm.features_selected = [0,5]        # ['left_paw',          0
                                     #  'right_paw',         1
                                     #  'nose',              2
                                     #  'jaw',               3 
                                     #  'right_ear',         4 
                                     #  'tongue',            5
                                     #  'lever',             6
                                     
                                     #'all',                 7   NOT CODED
                                     #'code_04',             8
                                     #'code_04_lockout']     9

#                                        
svm.inter_feature_lockout_window = 0
                                        

# can add this to aut
svm.get_sessions()
#
print (svm.sessions)

#
svm.load_trials()

# convert data to train and test datasets;
svm.prepare_and_run_classifier()

# print (svm.acc)

# 
fig=plt.figure()
acc = np.array(svm.acc)
mean = acc.mean(1)
std = np.std(acc,1)
t = np.arange(mean.shape[0])/30.-14
plt.fill_between(t,mean+std, mean-std, alpha=.2)
plt.plot(t,mean)

# 
plt.ylim(bottom=0)
#plt.xlim(-5,5)
plt.plot([t[0],t[-1]],[1./len(svm.features_selected),1./len(svm.features_selected)],'--',c='black')
plt.title(labels[svm.features_selected[0]]+ " "+labels[svm.features_selected[1]])
plt.show()


['IJ2pm_Feb1_30Hz']
  running:  IJ2pm_Feb1_30Hz
Lever to [Ca] shift:  2.533333333333333
DLC to [Ca] shift:  -9.600000000000001
Trials stack (make sure same across D1):  (2, 38, 900, 30)
0  size:  (38, 900, 30)
1  size:  (38, 900, 30)


904it [01:53,  7.94it/s]                         


In [33]:
##########################################################
########## MOST SIGNIFICANT COEFFICIENT ANALYSIS #########
##########################################################

# print (len(svm.coef))
# print (np.array(svm.coef[1]).shape)
# ctr=1
# arr = []
# for k in range(0,len(svm.coef)-30,1):
# #     ax=plt.subplot(5,6,ctr)
#     temp = np.array(svm.coef[k]).squeeze().mean(0).reshape(30,30).T
#     arr.append(temp)

# # most significant analyais
# arr = np.array(arr)
# t = np.arange(arr.shape[0])/30-14
# for k in range(arr.shape[1]):
#     for p in range(arr.shape[2]):
#         plt.plot(t,arr[:,k,p],alpha=.7)
# plt.show()

In [5]:
# ###############################################
# ############ LEVER TO BODY CORRELATION ########
# ###############################################
# import pycorrelate
# t = svm.code_04_times_unshifted[:,1]

# fig=plt.figure()
# bins = np.arange(-10,10,1/15)
# for k in range(7):
#     ax=plt.subplot(3,3,k+1)
#     u =  np.array(svm.features_unshifted[k])[:,1]
#     res = pycorrelate.pcorrelate(t, u, bins=bins)

#     plt.plot(bins[:-1],res)
#     argmax = np.argmax(res)
#     print (" shift: ", bins[argmax])
#     plt.title(labels[k]+ " "+str(round(bins[argmax],2))+"sec")
# plt.suptitle("Correlation between body and lever ")
# plt.show()


 shift:  -9.733333333333334
 shift:  -9.600000000000001
 shift:  0.5999999999999623
 shift:  -9.866666666666667
 shift:  3.5999999999999517
 shift:  -9.933333333333334
 shift:  3.799999999999951


In [6]:
##############################################
############ BODY TO BODY CORRELATION ########
##############################################
fig=plt.figure()
bins = np.arange(-10,10,.1)
ctr=1
bins = np.arange(-10,10,1/15.)
for k in range(7):
    for p in range(8):
        ax=plt.subplot(7,8,ctr)
        t = np.array(svm.features_unshifted[k])[:,1]
        if p==7:
            u =  code_04[:,1]  
        else:
            u =  np.array(svm.features_unshifted[p])[:,1]
        res = pycorrelate.pcorrelate(t, u, bins=bins)

        plt.plot(bins[:-1],res)
        argmax = np.argmax(res)

        if k==0:
            plt.title(labels[p]+ " " + str(round(bins[argmax],2))+"sec",fontsize=10,pad=.9)
        else:
            plt.title(str(round(bins[argmax],2))+"sec",fontsize=10,pad=.9)
            
            
        if p==0:
            plt.ylabel(labels[k],fontsize=10)
        if k!=6:
            plt.xticks([])
        plt.yticks([])
        
        std = np.std(res)
        plt.plot([bins[0],bins[-1]],[std,std],'--',c='black')
        
        ctr+=1
        
plt.suptitle("Correlation between body and lever ")
plt.show()


NameError: name 'code_04' is not defined

In [8]:

    
    
# 


IA1pm_Feb1_30Hz
Lever to [Ca] shift:  2.566666666666667
DLC to [Ca] shift:  -4.733333333333352
IA1pm_Feb2_30Hz
Lever to [Ca] shift:  2.5
DLC to [Ca] shift:  -9.8
IA1pm_Feb3_30Hz
Lever to [Ca] shift:  2.033333333333333
DLC to [Ca] shift:  4.1999999999999496
IA1pm_Feb4_30Hz
Lever to [Ca] shift:  2.4
DLC to [Ca] shift:  -10.0
IA1pm_Feb5_30Hz
Lever to [Ca] shift:  2.2666666666666666
DLC to [Ca] shift:  -9.533333333333335
IA1pm_Feb9_30Hz
Lever to [Ca] shift:  2.3333333333333335
DLC to [Ca] shift:  2.7999999999999545
IA1pm_Feb10_30Hz
Lever to [Ca] shift:  2.7333333333333334
DLC to [Ca] shift:  -4.466666666666686
IA1pm_Feb11_30Hz
Lever to [Ca] shift:  2.533333333333333
DLC to [Ca] shift:  4.733333333333281
IA1pm_Feb12_30Hz
Lever to [Ca] shift:  2.466666666666667
DLC to [Ca] shift:  -0.46666666666670054
IA1pm_Feb15_30Hz
Lever to [Ca] shift:  2.6666666666666665
DLC to [Ca] shift:  4.266666666666616
IA1pm_Feb16_30Hz
Lever to [Ca] shift:  2.5
DLC to [Ca] shift:  4.599999999999948
IA1pm_Feb17_30Hz

IJ2pm_Feb4_30Hz
Lever to [Ca] shift:  2.3
DLC to [Ca] shift:  3.9999999999999503
IJ2pm_Feb5_30Hz
Lever to [Ca] shift:  2.6666666666666665
DLC to [Ca] shift:  4.399999999999949
IJ2pm_Feb9_30Hz
Lever to [Ca] shift:  2.2333333333333334
DLC to [Ca] shift:  4.066666666666617
IJ2pm_Feb10_30Hz
Lever to [Ca] shift:  2.1666666666666665
DLC to [Ca] shift:  3.666666666666618
IJ2pm_Feb11_30Hz
Lever to [Ca] shift:  2.6666666666666665
DLC to [Ca] shift:  4.599999999999948
IJ2pm_Feb12_30Hz
Lever to [Ca] shift:  2.433333333333333
DLC to [Ca] shift:  4.599999999999948
IJ2pm_Feb15_30Hz
Lever to [Ca] shift:  2.533333333333333
DLC to [Ca] shift:  4.266666666666616
IJ2pm_Feb16_30Hz
Lever to [Ca] shift:  3.2
DLC to [Ca] shift:  5.333333333333279
IJ2pm_Feb17_30Hz
IJ2pm_Feb18_30Hz
IJ2pm_Feb22_30Hz
IJ2pm_Feb23_30Hz
IJ2pm_Feb24_30Hz
IJ2pm_Feb25_30Hz
IJ2pm_Feb26_30Hz
IJ2pm_Feb29_30Hz
IJ2pm_Mar1_30Hz
IJ2pm_Mar2_30Hz
IJ2pm_Mar3_30Hz
IJ2am_Mar7_30Hz
IJ2pm_Mar8_30Hz
IJ2am_Mar9_30Hz
IJ2am_Mar10_30Hz
IJ2am_Mar11_30Hz


0 0 1 [ 1.33333333e-01 -3.55271368e-14 -3.55271368e-14 -3.55271368e-14
 -3.55271368e-14 -3.55271368e-14 -3.55271368e-14 -3.55271368e-14
 -2.66666667e-01 -3.55271368e-14 -3.55271368e-14 -3.55271368e-14
 -3.55271368e-14 -3.55271368e-14 -3.55271368e-14 -3.55271368e-14
 -3.55271368e-14 -3.55271368e-14 -3.55271368e-14 -3.55271368e-14
 -3.55271368e-14 -3.55271368e-14 -3.55271368e-14 -3.55271368e-14
 -3.55271368e-14 -3.55271368e-14 -3.55271368e-14  1.33333333e-01
 -3.55271368e-14 -6.66666667e-02]

0 0 2 [-2.66666667e-01 -3.55271368e-14 -6.66666667e-01 -2.66666667e-01
 -4.00000000e-01  2.66666667e-01 -2.00000000e-01  7.33333333e-01
 -2.66666667e-01 -2.66666667e-01 -2.66666667e-01 -1.33333333e-01
 -3.55271368e-14  8.66666667e-01 -1.33333333e-01 -3.55271368e-14
  2.66666667e-01  2.66666667e-01 -3.55271368e-14 -3.55271368e-14
 -1.33333333e-01  2.66666667e-01 -3.55271368e-14 -3.55271368e-14
 -3.55271368e-14 -1.33333333e-01 -3.55271368e-14 -3.55271368e-14
  1.33333333e-01]

0 0 3 [-3.55271368e-14 -

1 1 2 [-1.01923237 -3.52651398  0.12005946 -0.10683464 -0.75279353 -0.0253124
  0.36311001  0.30762951 -0.40021119 -0.75904954  0.09374976  0.11530218]

1 1 3 [-0.27441392 -0.90473067 -2.90962118  0.01503799 -0.37417175 -0.39644732
 -0.42632078 -0.11909424  0.40353845 -2.15570916  0.38519265 -1.76226368]

1 1 4 [-0.20998118 -0.37057944 -0.44379132 -0.18056835 -0.07495841 -0.24926844
 -0.12897531  0.05088769 -0.17129844 -0.49799539 -0.22302022]

1 1 5 [-0.25798218 -0.85351558 -2.80529177 -0.36813212  0.03942286 -0.25346105
  0.04150899  0.27282508  0.05942363 -2.3684086   0.03849474 -2.25401303]

1 1 6 [-0.64654654  2.54120454 -0.38663346  0.12050109  0.32944867 -0.33804299
  0.11361698  0.47106383 -0.47942764  0.68607449 -1.04610972]

1 2 3 [-6.59681616e-01 -1.78566630e+00  5.51851944e-02 -2.53453698e-02
  1.44348877e-01 -4.33165678e-04  7.42453222e-02 -1.11350440e-01
 -2.19375213e-01 -4.50789432e-01  1.72020484e-01  1.27293452e-01]

1 2 4 [-0.96681269 -0.1693804   0.27351162  0.579926

 -0.13333333 -2.53333333  0.26666667 -2.13333333]

4 2 6 [-2.13333333e+00  2.00000000e-01 -3.55271368e-14  2.00000000e-01
  2.66666667e-01  6.66666667e-02 -4.66666667e-01 -2.66666667e-01
 -2.40000000e+00]

4 3 4 [ 1.33333333e-01 -2.66666667e-01 -3.55271368e-14  1.26666667e+00
  6.66666667e-02  2.00000000e-01 -3.55271368e-14 -3.55271368e-14
  6.66666667e-02 -2.00000000e-01 -2.66666667e-01]

4 3 5 [0.33333333 0.13333333 0.13333333 0.13333333 0.13333333 1.26666667
 0.66666667 0.46666667 0.26666667 1.        ]

4 3 6 [-3.55271368e-14 -2.73333333e+00 -3.55271368e-14 -1.33333333e-01
  4.66666667e-01 -1.33333333e-01 -1.33333333e-01 -2.66666667e-01
  2.00000000e-01 -1.53333333e+00]

4 4 5 [-3.06666667e+00  2.66666667e-01 -3.55271368e-14 -2.66666667e-01
 -3.55271368e-14  6.00000000e-01  5.33333333e-01  3.33333333e-01
  3.33333333e-01 -3.13333333e+00]

4 4 6 [-2.86666667e+00 -4.00000000e-01  4.66666667e-01 -1.33333333e-01
 -3.55271368e-14 -1.33333333e-01 -3.55271368e-14  1.33333333e-01
 -3.55271

In [210]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

def diagonalize(data):
    for k in range(data.shape[0]):
        for p in range(0,k,1):
            data[k,p]=np.nan
    return data
            
fig = plt.figure(figsize=(12,12))
ax=plt.subplot(111)
data = correlated[:-1,:-1].astype('float32')
data = diagonalize(data)


data2 = uncorrelated[:-1,:-1].astype('float32')
data2 = diagonalize(data2)

im = ax.imshow(data/(data+data2), extent = [0,6,0,6])

# create an axes on the right side of ax. The width of cax will be 5%
# of ax and the padding between cax and ax will be fixed at 0.05 inch.
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)

plt.colorbar(im, cax=cax)

ax.set_xticks(np.arange(6)+0.5)
ax.set_xticklabels(labels[1:7],rotation=45,fontsize=20)

ax.set_yticks(np.arange(6)+0.5)
ax.set_yticklabels(labels[0:6],rotation=0,fontsize=20)



# # 
# ax=plt.subplot(122)
# data = uncorrelated[:-1,:-1].astype('float32')
# data = diagonalize(data)
# im = ax.imshow(data, extent = [0,6,0,6])

# # create an axes on the right side of ax. The width of cax will be 5%
# # of ax and the padding between cax and ax will be fixed at 0.05 inch.
# divider = make_axes_locatable(ax)
# cax = divider.append_axes("right", size="5%", pad=0.05)

# ax.set_xticks(np.arange(6)+0.5)
# ax.set_xticklabels(labels[1:7],rotation=45,fontsize=12)


#ax.xaxis.set_xticks(xticks)

plt.colorbar(im, cax=cax)

if True:
    plt.savefig('/home/cat/correlated_bodyparts_matrices.png',dpi=100)
    plt.close()
else:
    plt.show() 
    

In [202]:
fig = plt.figure()
vmin = np.min(rejected)
vmax = np.max(rejected)
print (vmin,vmax)
plt.imshow(rejected)
plt.show()

0 23


In [ ]:
fname = '/home/cat/concatenated_trials.npy'
svm  = PredictSVMTime()
svm.fname = fname

In [29]:
svm.process_data2()

X_l:  (196, 900, 20)
X_l:  (22, 900, 20)
X_l:  (196, 900, 20)
X_l:  (22, 900, 20)
X_l:  (196, 900, 20)
X_l:  (22, 900, 20)
X_l:  (196, 900, 20)
X_l:  (22, 900, 20)
X_l:  (196, 900, 20)
X_l:  (22, 900, 20)
X_l:  (196, 900, 20)
X_l:  (22, 900, 20)
X_l:  (196, 900, 20)
X_l:  (22, 900, 20)
X_l:  (196, 900, 20)
X_l:  (22, 900, 20)
X_l:  (197, 900, 20)
X_l:  (21, 900, 20)
X_l:  (197, 900, 20)
X_l:  (21, 900, 20)


In [30]:
data = np.load('/home/cat/concatenated_res.npz')

cf = data['conf_matrix_ten']
print (cf.shape)

(10, 30, 30)


In [22]:
res = cf.mean(0)

plt.imshow(res)

plt.show()

In [ ]:
######################################################
##### PREDICT SVM DECISION CHOICE SINGLE SESSION #####
######################################################

# select animal names
names = ['IA1','IA2','IA3','IJ1','IJ2','AQ2'] # "AR4" and other datasets could work
#names = ['IA1']

# 
svm = PredictSVMChoice()
svm.root_dir = data_dir
svm.random_flag = False  # shuffle data to show baseline
svm.verbose = True

# window parameters
svm.window = 15              # prediction window, backwards in time in seconds
svm.lockout_window = 10      # time for locking out other pulls; OPTIONAL
svm.sliding_window = 30      # number of frames in sliding window for SVM steps; usually 1 sec = 30 frames
svm.lockout = False

# movement related params
svm.no_movement = 3          # number of seconds of quiescence before behavior initiation; parameter comes from
                             #     the "generate_movements_quiescent_periods" notebook in behavior_analysis
svm.random_lockout = 3       # number of seconds of buffer for selecting random trials for control

# pca params
svm.pca_flag = True
svm.pca_var = 0.95      # this is total amount of var explained to be used - NOT USED FOR NOW
svm.all_comps = 30      # this is the total # components saved
svm.nComp = 30          # this is the total # of comps to be used for decoding
            
# svm parameters
svm.min_trials = 10     # minimum number of trials to run SVM
svm.xvalidation = 10    # KFold xvalidation step
svm.data_split = 0.8    # split 80/20 data for training  NOT USED WITH XVALIDATION WHICH SPLITS 90/10
svm.method = 'sigmoid'  # method used for computing SVM

# run-time parameters
svm.parallel = True
svm.n_cores = svm.xvalidation
svm.overwrite = False

## session info
# session_ids = ['Mar1_', 'Mar2_', 'Mar3_', 'Feb29', 'Mar7_']
# svm.session_id = 'all'
svm.session_id = 'all'

# 
# codes = ['left_paw','right_paw','jaw']
labels = ['left_paw', 
          'right_paw',
          'nose',
          'jaw',
          'right_ear',
          'tongue',
          'lever',
          'all',
          'code_04',
          'code_04_lockout']

svm.labels = labels
svm.code = '_whole_stack'

# 
for name in names:
    svm.animal_id = name

    #
    svm.predict_whole_stack()   # this requires PCA to be run on the entire imaging session first: 400000, 128, 128
                                        # need also movement initiations 
                                        # need also shift between initiations and ca imaging
                                # This bypasses the need to compute [Ca] stms for each behavior/trial


In [5]:
##############################
##### VISUALIZE A RESULT #####
##############################

import pandas as pd

#
fname = '/media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_IA1pm_Feb2_30Hz_whole_stack_pca30Components_window15sec_Xvalid10_Slidewindow30Frames_accuracy.pk'
accuracy = np.array(pd.read_pickle(fname))

# 
fig = plt.figure()
for k in range(len(accuracy)):
    ax=plt.subplot(2,5,k+1)
    acc = np.array(accuracy[k])
    print (acc.shape)

    if acc.shape[0]==0:
        continue
    t = np.arange(acc.shape[0])/30-14
    
    # 
    mean = acc.mean(1)
    plt.plot(t,mean,label=labels[k])
    std = np.std(acc)
    ax.fill_between(t, mean+std,
                   mean-std,
                  alpha=.2)

    # 
    plt.xlim(-14,15)
    plt.xlim(-10,10)
    plt.ylim(0.4,1.0)
    plt.plot([-14,15],[0.5,0.5],'--',c='black')
    
    plt.legend()
plt.show()

<ipython-input-5-d794d1f35346>:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  accuracy = np.array(pd.read_pickle(fname))


(870, 10)
(870, 10)
(870, 10)
(870, 10)
(870, 10)
(870, 10)
(0,)
(870, 10)
(870, 10)
(870, 10)


In [12]:
# ######################################################
# ######## SET SHIFTS FOR LEVER PULL AND DLC DATA ######
# ######################################################

# # SET SHIFT FOR LEVER PULL
# # Find blue light on/off
# blue_light_threshold = 400  #Intensity threshold; when this value is reached - imaging light was turned on
# start_blue = 0; end_blue = len(aligned_images)

# if np.average(aligned_images[0])> blue_light_threshold:    #Case #1: imaging starts with light on; need to remove end chunk; though likely bad recording
#     for k in range(len(aligned_images)):
#         if np.average(aligned_images[k])< blue_light_threshold:
#             #self.aligned_images = self.aligned_images[k:]
#             end_blue = k
#             break
# else:                                                           #Case #2: start with light off; remove starting and end chunks;
#     #Find first light on
#     for k in range(len(aligned_images)):
#         if np.average(aligned_images[k])> blue_light_threshold:
#             start_blue = k
#             break

#     #Find light off - count backwards from end of imaging data
#     for k in range(len(aligned_images)-1,0,-1):
#         if np.average(aligned_images[k])> blue_light_threshold:
#             end_blue= k
#             break

# #
# filtered_filename = images_file[:-4]+'_'+selected_dff_filter+'_'+str(low_cut)+'hz_'+str(high_cut)+'hz.npy'
# if os.path.exists(filtered_filename):
#     try:
#         aligned_images = np.load(filtered_filename, allow_pickle=True)
#     except:
#         print ("aligned filtered images corrupt... recomputing: ", filtered_filename)
#         self.filter_data(root_dir, recording)
#         aligned_images = np.load(filtered_filename)
# else:
#     print ("aligned filtered images missing... recomputing: ", filtered_filename)
#     self.filter_data(root_dir, recording)
#     aligned_images = np.load(filtered_filename)

# aligned_images = aligned_images[start_blue:end_blue]


        
        

(71, 15, 901)


In [3]:
#############################################################
##################VISUALIZe ################
#############################################################
data=[]
#data.append(np.load('/mnt/f418adc0-4ff9-410c-93c9-d6d21a1e9c35/yuki/IJ2/SVM_Scores/SVM_Scores_IJ2pm_Mar3_30Hz_code_04_trial_ROItimeCourses_15sec_pca_0.95_10xvalid.npy').T)
#data.append(np.load('/mnt/f418adc0-4ff9-410c-93c9-d6d21a1e9c35/yuki/IJ2/SVM_Scores/SVM_Scores_IJ2pm_Mar3_30Hz_code_04_trial_ROItimeCourses_15sec_pca_0.95_50xvalid.npy').T)
#data.append(np.load('/mnt/f418adc0-4ff9-410c-93c9-d6d21a1e9c35/yuki/IJ2/SVM_Scores/SVM_Scores_IJ2pm_Mar3_30Hz_code_04_trial_ROItimeCourses_15sec_pca_0.95_100xvalid.npy').T)

fnames = [
    #'/mnt/f418adc0-4ff9-410c-93c9-d6d21a1e9c35/yuki/IA1/SVM_Scores/IA1pm_Mar3_30Hz/SVM_Scores_IA1pm_Mar3_30Hz_code_04_trial_ROItimeCourses_15sec_pca_0.95_Xvalid10_Slidewindow30.npy',
    '/mnt/f418adc0-4ff9-410c-93c9-d6d21a1e9c35/yuki/IA1/SVM_Scores/IA1pm_Mar3_30Hz/SVM_Scores_IA1pm_Mar3_30Hz_code_04_trial_ROItimeCourses_15sec_pca_0.95_Xvalid100_Slidewindow30.npy',
    '/mnt/f418adc0-4ff9-410c-93c9-d6d21a1e9c35/yuki/IA1/SVM_Scores/IA1pm_Mar3_30Hz/SVM_Scores_IA1pm_Mar3_30Hz_code_04_trial_ROItimeCourses_15sec_pca_0.95_Xvalid100_Slidewindow90.npy',
    #     '/mnt/f418adc0-4ff9-410c-93c9-d6d21a1e9c35/yuki/IA1/SVM_Scores/IA1pm_Mar3_30Hz/SVM_Scores_IA1pm_Mar3_30Hz_code_04_trial_ROItimeCourses_15sec_pca_0.95_Xvalid20_Slidewindow30.npy'
    #'/mnt/f418adc0-4ff9-410c-93c9-d6d21a1e9c35/yuki/IA1/SVM_Scores/IA1pm_Mar3_30Hz/SVM_Scores_IA1pm_Mar3_30Hz_code_04_trial_ROItimeCourses_15sec_pca_0.95_Xvalid100_Slidewindow10.npy'

]
# 
clrs=['black','blue','red']
ctr=0
fig=plt.figure()
for fname in fnames:
    data = np.load(fname).T
    
    sd = np.where(data>1)
    print ("sd: ", sd)
    
    idx = fname.find('Xvalid')
    idx2 = fname[idx:].find("_")
    xvalid = int(fname[idx+6:idx+idx2])
    print ("xvalid: ", xvalid)
    
    idx = fname.find('Slidewindow')
    idx2 = fname[idx:].find(".npy")
    swindow = int(fname[idx+11:idx+idx2])
    print ("swindow: ", swindow)
    
    
    
    #ax=plt.subplot(2,2,k+1)
    mean = data.mean(1)
    std = np.std(data,1)
    t=(np.arange(mean.shape[0]))/30-15.+swindow/30
    plt.plot(t,mean,c=clrs[ctr])
    plt.fill_between(t,mean+std, mean-std, color=clrs[ctr], alpha=.2,
                    label="xvalid: "+str(xvalid)+
                          '  swindow: '+str(swindow),
                    )
    
    plt.plot([t[0],t[-1]],[0.5,0.5],'--',c='black')
    plt.legend()
    plt.ylim(0.3,1.0)
    ctr+=1
plt.xlim(right=0)
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/f418adc0-4ff9-410c-93c9-d6d21a1e9c35/yuki/IA1/SVM_Scores/IA1pm_Mar3_30Hz/SVM_Scores_IA1pm_Mar3_30Hz_code_04_trial_ROItimeCourses_15sec_pca_0.95_Xvalid100_Slidewindow30.npy'

In [4]:
#####################################
##### PREDICT SVM DECISION TIME #####
#####################################
svm = PredictSVMTime()
svm.main_dir = '/media/cat/4TBSSD/yuki/'

# default params
svm.lockout = False
svm.lockout_window = 10
svm.pca_flag = True
svm.pca_var = 0.95
svm.window = 10
svm.use_saved_model = False
svm.fname_saved_model = '/media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Feb11_30Hz/analysis/00599_svm.pkl'

# 
names = ['IA1','IA2','IA3','IJ1','IJ2','AQ2'] # "AR4" and other datasets could work

# 
lockouts = [False, True]
for lockout in lockouts:
    svm.lockout=lockout
    for id_ in names:
        svm.animal_id = id_
        svm.session_id = 'all'

        # 
        svm.predict()

# svm.process_data()

  0%|          | 0/71 [00:00<?, ?it/s]

 too few trials... skipping


  4%|▍         | 3/71 [00:01<00:27,  2.46it/s]

no such filename, skipping
 too few trials... skipping


 14%|█▍        | 10/71 [00:02<00:13,  4.64it/s]

...

 30%|███       | 33/110 [00:05<00:19,  3.93it/s]

no such filename, skipping
no such filename, skipping
no such filename, skipping
no such filename, skipping
 too few trials... skipping
no such filename, skipping
no such filename, skipping
 too few trials... skipping
 too few trials... skipping
no such filename, skipping
no such filename, skipping
no such filename, skipping
no such filename, skipping
 too few trials... skipping


 62%|██████▏   | 68/110 [00:14<00:21,  1.97it/s]

no such filename, skipping


100%|██████████| 110/110 [00:35<00:00,  3.07it/s]


In [18]:
################################################
################################################
################################################
fig = plt.figure()
svm.animal_id = 'AQ2'
svm.session_id = 'all'
svm.get_sessions()
print (svm.sessions)

#
ctr=1
for session_id in svm.sessions:
    ax=plt.subplot(8,10,ctr)
    svm.session_id = session_id
    try:
        svm.plot_decision_time(ax)
        plt.title(session_id+", #: "+str(svm.n_trials), fontsize=6)
        if ctr!=1:
            plt.yticks([])
        if ctr!=31:
            plt.xticks([])

        ctr+=1

    except:
        pass
plt.suptitle(svm.animal_id,fontsize=20)

['AQ2am_Dec9_30Hz' 'AQ2am_Dec10_30Hz' 'AQ2pm_Dec10_30Hz'
 'AQ2am_Dec11_30Hz' 'AQ2pm_Dec14_30Hz' 'AQ2am_Dec14_30Hz'
 'AQ2pm_Dec16_30Hz' 'AQ2am_Dec17_30Hz' 'AQ2pm_Dec17_30Hz'
 'AQ2am_Dec18_30Hz' 'AQ2pm_Dec18_30Hz' 'AQ2am_Dec21_30Hz'
 'AQ2am_Dec22_30Hz' 'AQ2am_Dec23_30Hz' 'AQ2am_Dec28_30Hz'
 'AQ2am_Dec29_30Hz' 'AQ2am_Dec30_30Hz' 'AQ2am_Dec31_30Hz'
 'AQ2am_Jan4_30Hz' 'AQ2am_Jan5_30Hz' 'AQ2am_Jan6_30Hz' 'AQ2am_Jan7_30Hz'
 'AQ2am_Jan8_30Hz' 'AQ2pm_Jan11_30Hz' 'AQ2am_Jan11_30Hz'
 'AQ2pm_Jan12_30Hz' 'AQ2am_Jan12_30Hz' 'AQ2pm_Jan13_30Hz'
 'AQ2am_Jan13_30Hz' 'AQ2pm_Jan14_30Hz' 'AQ2am_Jan14_30Hz'
 'AQ2am_Jan15_30Hz' 'AQ2pm_Jan15_30Hz' 'AQ2pm_Jan18_30Hz'
 'AQ2am_Jan18_30Hz' 'AQ2am_Jan19_30Hz' 'AQ2pm_Jan19_30Hz'
 'AQ2pm_Jan20_30Hz' 'AQ2am_Jan20_30Hz' 'AQ2pm_Jan21_30Hz'
 'AQ2am_Jan21_30Hz' 'AQ2pm_Jan22_30Hz' 'AQ2am_Jan22_30Hz'
 'AQ2am_Jan25_30Hz' 'AQ2pm_Jan25_30Hz' 'AQ2pm_Jan26_30Hz'
 'AQ2am_Jan26_30Hz' 'AQ2am_Jan27_30Hz' 'AQ2pm_Jan27_30Hz'
 'AQ2am_Jan28_30Hz' 'AQ2pm_Jan28_30Hz' 'AQ2am_Jan29_30Hz'
 

Text(0.5, 0.98, 'AQ2')

In [ ]:
names = ['IA1','IA2','IA3','IJ1','AQ2'] # "AR4" and other datasets could work

for name in names:
    rnn.animal_id = name
    rnn.session_id = 'all'

    rnn.run_rnn()

In [12]:
#
data1 = np.load('/media/cat/4TBSSD/yuki/IJ2/RNN_scores/Mar3_200_0.0001.npy.npz')
data1 = np.load('/media/cat/4TBSSD/yuki/IJ2/RNN_scores/IJ2pm_Mar2_30Hz_1000_0.0001.npz')
#data1 = np.load('/media/cat/4TBSSD/yuki/IJ2/RNN_scores/IJ2pm_Feb29_30Hz_1000_0.0001.npz')
b = data1['b_rnn']
c = data1['c_s']

#
print (b.shape)
mean = b.mean(1)
t=np.linspace(-9.5, 0, mean.shape[0])

ax=plt.subplot(111)

plt.plot(t, mean, c='black')
plt.fill_between(t, mean+c, mean-c, color='black', alpha=.2,
                label='RNN - 1000epochs')

#
data1 = np.load('/media/cat/4TBSSD/yuki/IJ2/RNN_scores/IJ2pm_Mar2_30Hz_200_0.0001.npz')
#data1 = np.load('/media/cat/4TBSSD/yuki/IJ2/RNN_scores/IJ2pm_Feb29_30Hz_200_0.0001.npz')
b = data1['b_rnn']
c = data1['c_s']

#
print (b.shape)
mean = b.mean(1)
t=np.linspace(-9.5, 0, mean.shape[0])
plt.plot(t, mean, c='red')
plt.fill_between(t, mean+c, mean-c, color='red', alpha=.2,
                label='RNN - 200epochs')
#
if False:
    fname_svm = '/media/cat/4TBSSD/yuki/IJ2/SVM_Scores/SVM_Scores_IJ2pm_Mar2_30Hz_code_04_trial_ROItimeCourses_10sec_pca_0.95.npy'
    data2 = np.load(fname_svm)
    print (data2.shape)
    data2 = data2[:300]
    mean = data2.mean(1)
    std = data2.std(1)
    t= np.linspace(t[0],t[-1],mean.shape[0])
    plt.plot(t, mean, 'blue')
    plt.fill_between(t, mean+std, mean-std, color='blue', alpha=.2,
                    label='SVM - 2 frames sliding window')

plt.legend(fontsize=20)
plt.xlim(-10,0)
plt.ylim(0.4, 1.0)
plt.plot([-10,0],[0.5,0.5],'r--',c='black')
ax.tick_params(axis='both', which='major', labelsize=20)
#plt.suptitle(os.path.split(fname_svm)[1])
plt.xlabel("Time (sec)",fontsize=20)
plt.ylabel("Decoding accuracy", fontsize=20)

plt.show()

(10, 10)
(10, 10)


<ipython-input-12-e7b69b6f7608>:48: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "r--" (-> color='r'). The keyword argument will take precedence.
  plt.plot([-10,0],[0.5,0.5],'r--',c='black')


In [ ]:
OLDER CODE BELOW

In [11]:
#############################################
############## VISUALIZE RESULTS ############
#############################################
    
# LEVER PULL
vis = Visualize()

# lever-related data
vis.main_dir = data_dir
vis.random_flag = False  # shuffle data to show baseline

# 
vis.window = 10
vis.lockout_window = 10
vis.code = 'code_04'
vis.lockout = False

vis.animal_id = "IA1"
vis.session_id = "Feb23_"
title = vis.animal_id + "  "+vis.session_id

#############################################
############## DECISION TYPE ################
#############################################
# select animal and session

# load pca 
fig =plt.figure()

# ax=plt.subplot(2,3,1)
# vis.pca_flag = False
# vis.lockout = True
# vis.pca_var = 0.99
# vis.plot_decision_choice('black',str(vis.pca_var),title, ax)

# vis.lockout = True
# vis.plot_decision_choice('blue',str(vis.pca_var)+" lockout",title, ax)

# 
# vis.lockout = True
# vis.plot_decision_choice('blue',str(vis.pca_var) + " lockout",title, ax)


# 
vars_ = [0.95]
for k in range(len(vars_)):
    #ax=plt.subplot(2,3,k+2)
    ax=plt.subplot(1,1,k+1)
    vis.pca_flag = True
    vis.lockout = False
    vis.pca_var = vars_[k]
    vis.plot_decision_choice('black',str(vis.pca_var),title, ax)

    # 
    vis.pca_flag = True
    vis.lockout = True
    vis.plot_decision_choice('blue',str(vis.pca_var)+" lockout",title, ax)

# vis.pca_flag = True
# vis.pca_var = 0.99
# ax=plt.subplot(2,3,2)
# vis.plot_decision_choice('blue','xxpcs_0.99Variance',title, ax)


# vis.pca_flag = True
# vis.pca_var = 0.95
# ax=plt.subplot(2,3,3)
# vis.plot_decision_choice('red','8pcs_0.95Variance',title, ax)

# vis.pca_flag = True
# vis.pca_var = 0.90
# ax=plt.subplot(2,3,4)
# vis.plot_decision_choice('green','4pcs_0.90Variance',title, ax)


# vis.pca_var = 0.50
# ax=plt.subplot(2,3,5)
# vis.plot_decision_choice('magenta','2pcs_0.50Variance',title, ax)


# fname = vis.main_dir+'/'+animal_id+'/SVM_scores_'+animal_id+'_lockout_'+str(session)+'.npy'
# vis.load_data(fname)
# vis.plot_decision_choice('blue','lockout-denoised')


Self session: IA1pm_Feb23_30Hz
Self session: IA1pm_Feb23_30Hz


In [7]:
import pickle as pk

file = open('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/IA1pm_Feb3_30Hz_code_04_lockout_10sec_trial_ROItimeCourses_10sec_pca.pkl', 'rb')

# dump information to that file
pca = pk.load(file)

print (pca.components_.shape)


(1500, 16384)


In [9]:
data_stm = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/IA1pm_Feb3_30Hz_10.0sec_butterworth_globalAverage_0.1hz_6.0hz_04code_stm.npy')
print (data_stm.shape)

(62, 601, 128, 128)


In [10]:
# 
X = data_stm.reshape(data_stm.shape[0]*data_stm.shape[1],
                     data_stm.shape[2]*data_stm.shape[3])

mu= np.mean(X, axis=0)


NameError: name 'nComp' is not defined

In [11]:
nComp = 9
Xnew = np.dot(pca.transform(X)[:,:nComp],
             pca.components_[:nComp,:])

Xnew+=mu

data_stm_denoised_reshaped = Xnew.reshape(data_stm.shape[0], data_stm.shape[1],
                                          data_stm.shape[2], data_stm.shape[3])


In [17]:
time_filters = pca.transform(X)[:,:nComp]
pca_time_filters_only = time_filters.reshape(data_stm.shape[0], data_stm.shape[1],-1).transpose(0,2,1)
print (pca_time_filters_only.shape)


(62, 9, 601)


(62, 601, 9)


In [21]:
data = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb23_30Hz/IA1pm_Feb23_30Hz_code_04_trial_ROItimeCourses_10sec_pca_0.5.npy')
print (data.shape)

(82, 2, 601)


In [76]:
# -*- coding: utf-8 -*-
"""Cat Data.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1r-NABuBJOITVlW03VvhsQivPNzmNp4yp
"""

import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold 
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn import svm
import pandas as pd
from sklearn.metrics import confusion_matrix  
import random
from scipy import stats
from matplotlib.pyplot import MultipleLocator
import os


jobid = os.getenv('SLURM_ARRAY_TASK_ID')
name='IA1_lockout'

data_leverpress = np.load('/media/cat/4TBSSD/yuki/temp/IA1/data_04.npy',allow_pickle=True)[0]
data_random = np.load('/media/cat/4TBSSD/yuki/temp/IA1/data_04_random.npy',allow_pickle=True)[0]

print (data_leverpress.shape)


(62, 35, 601)


In [77]:


#classification of time (10-class)
X_l=data_leverpress #  [n_trials, n_areas, n_time_steps]
X_l=X_l.transpose((0,2,1))


X_R_l= X_l.reshape(-1,X_l.shape[1]*X_l.shape[2])
normal_X_l = preprocessing.normalize(X_R_l)
n_X_l=normal_X_l.reshape(X_l.shape[0],X_l.shape[1],X_l.shape[2])
X_l=n_X_l

X_l=X_l[:,:300,:]
X_k=X_l[:,0:30,:]
X_k=X_k.reshape(X_k.shape[0],X_k.shape[1]*X_k.shape[2])
for i in range(30,271,30):
    X_t = X_l[:,i:i+30,:]
    X_t=X_t.reshape(X_t.shape[0],X_t.shape[1]*X_t.shape[2])
    X_k=np.concatenate((X_k,X_t),axis=0)

y_ct=np.zeros(data_leverpress.shape[0])
for i in range(1,10):
    Z_ct=i*np.ones(data_leverpress.shape[0])
    y_ct=np.concatenate((y_ct,Z_ct))

# 
X_tSVM=X_k # 10s * 30frams
y_tSVM=y_ct # labels

#10-fold confusion matrix
clf = svm.SVC() # Non-linear classifier
ten_svm=[]
conf_matrix_ten=[]
kf = KFold(n_splits=10,random_state=None, shuffle=True) 
for train_index, test_index in kf.split(X_tSVM):
    X_train_k, X_test_k = X_tSVM[train_index], X_tSVM[test_index] 
    y_train_k, y_test_k = y_tSVM[train_index], y_tSVM[test_index]
    clf.fit(X_train_k, y_train_k)
    score=clf.score(X_test_k, y_test_k)
    y_predicted=clf.predict(X_test_k)
    cm=confusion_matrix(y_test_k,y_predicted)
    confusion_m=cm.T # make each row be the prediction
    conf_matrix_norm = confusion_m.astype('float') / confusion_m.sum(axis=1)[:,np.newaxis] #calculate the precision
    conf_matrix_norm = np.nan_to_num(conf_matrix_norm)
    ten_svm.append(score)
    conf_matrix_ten.append(conf_matrix_norm)
np.save('conf_10_'+str(name)+'_'+str(jobid)+'.npy',conf_matrix_ten)
sc=np.mean(conf_matrix_ten,axis=0) # mean of confusion matrix
np.save('conf_avg_'+str(name)+'_'+str(jobid)+'.npy',sc)

<ipython-input-77-0c90c545bbff>:41: RuntimeWarning: invalid value encountered in true_divide
  conf_matrix_norm = confusion_m.astype('float') / confusion_m.sum(axis=1)[:,np.newaxis] #calculate the precision
<ipython-input-77-0c90c545bbff>:41: RuntimeWarning: invalid value encountered in true_divide
  conf_matrix_norm = confusion_m.astype('float') / confusion_m.sum(axis=1)[:,np.newaxis] #calculate the precision
<ipython-input-77-0c90c545bbff>:41: RuntimeWarning: invalid value encountered in true_divide
  conf_matrix_norm = confusion_m.astype('float') / confusion_m.sum(axis=1)[:,np.newaxis] #calculate the precision
<ipython-input-77-0c90c545bbff>:41: RuntimeWarning: invalid value encountered in true_divide
  conf_matrix_norm = confusion_m.astype('float') / confusion_m.sum(axis=1)[:,np.newaxis] #calculate the precision
<ipython-input-77-0c90c545bbff>:41: RuntimeWarning: invalid value encountered in true_divide
  conf_matrix_norm = confusion_m.astype('float') / confusion_m.sum(axis=1)[:,np

In [78]:

#Next is the plotting.

confusion_10=sc.diagonal()
np.save('diagonal_'+str(name)+'_'+str(jobid)+'.npy',confusion_10)
confusion_s=conf_matrix_ten

confusion_d=[]
for i in range(0,10):
    confusion_d.append(confusion_s[i].diagonal())
x_std=np.std(confusion_d,axis=0)/(10**0.5)
x_s=x_std


fig = plt.figure(figsize=(21.5,20), dpi=64, facecolor='white')
plt.xticks(fontsize=60)
plt.yticks(fontsize=60)
axes = plt.subplot(111)
axes.tick_params(axis ='both', which ='both', length = 10,width=4,pad=20)

bwith = 3 
ax = plt.gca()
ax.spines['bottom'].set_linewidth(bwith)
ax.spines['left'].set_linewidth(bwith)
ax.spines['top'].set_linewidth(bwith)
ax.spines['right'].set_linewidth(bwith)
l=15

Y=np.linspace(0,1,12)
X=np.ones(Y.size)
x=np.arange(-9.5,0.5,1)
plt.text(0.1, 0.2, 'Lever Pull', fontdict={'size': 70}, rotation=90)
plt.text(-12.95, 1/10, 'Chance', fontdict={'size': 60}, rotation=0)
plt.axhline(y=1/10, color='black', linestyle='--',linewidth=6)

plt.errorbar(x,confusion_10,x_s,c='red',alpha=0.6,marker='s', mec='red', ms=3, mew=3,label='SVM',linewidth=l,elinewidth=12)

plt.plot( (0+0)*X, Y, color='black',ls='--',linewidth=6)

plt.title('Decoding decision time',fontdict={'size': 110},pad=60)
plt.xlabel('Time before Lever Pull (s)',fontdict={'size': 110},labelpad=60)
plt.ylabel('Decoding Accuracy',fontdict={'size': 110},labelpad=180)
plt.ylim(ymax = 1.09)
ax=plt.gca()
ax.xaxis.set_major_locator(MultipleLocator(2))
ax.xaxis.set_minor_locator(MultipleLocator(1))
ax.yaxis.set_major_locator(MultipleLocator(0.2))
ax.yaxis.set_minor_locator(MultipleLocator(0.1))

plt.xlim(xmin = -10.5)
plt.xlim(xmax = 0.9)

for i in range(0,10):
    a=confusion_10[i]
    b=x_s[i]
    n=10
    s=b
    t=(a-(1/10))/s
    df=n-1
    p = (1 - stats.t.cdf(t,df=df))
    
    if (p > 0.01 and p <= 0.05):
        plt.plot(-9.5+i, confusion_10[i]+x_std[i]+0.06, '*',c='black', ms=20)
        
    if (p > 0.00001 and p <= 0.01):
        plt.plot(-9.5+i,confusion_10[i]+x_std[i]+0.06,'*',c='black', ms=20)
        plt.plot(-9.5+i,confusion_10[i]+x_std[i]+0.08,'*',c='black', ms=20)
    if  p <= 0.00001:
        plt.plot(-9.5+i,confusion_10[i]+x_std[i]+0.06,'*',c='black', ms=20)
        plt.plot(-9.5+i,confusion_10[i]+x_std[i]+0.08,'*',c='black', ms=20)
        plt.plot(-9.5+i,confusion_10[i]+x_std[i]+0.10,'*',c='black', ms=20)

plt.show()
#plt.savefig('./time_svm_'+str(jobid)+'.jpg')
#plt.close()

<ipython-input-78-40a74f02ec5b>:57: RuntimeWarning: divide by zero encountered in double_scalars
  t=(a-(1/10))/s


In [10]:
data= np.load('/media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Mar3_30Hz/IJ2pm_Mar3_30Hz_code_04_lockout_10sec_random_ROItimeCourses_10sec_pca_0.95_spatial.npy')
print (data.shape)

(7, 16384)


In [38]:

def make_video_dlc(traces,
                   movements,
                   fname_video,
                   start=0,
                   end=1000):
    
    ####################################################################
    ################## MAKE LABELED VIDEOS #############################
    ####################################################################
    import cv2
    from tqdm import trange
    
    
    clrs = ['blue','red','yellow','green', 'magenta','pink','cyan']


    #
    #traces = np.load(fname_traces)
    print ("Traces: ", traces.shape)

    #
    #fname_in = '/media/cat/4TBSSD/yuki_lever-ariadna-2020-07-21/IA1/videos_to_label/prestroke/IA1pm_Feb5_30Hz.mp4'
    original_vid = cv2.VideoCapture(fname_video)

    #
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)
    ret, frame = original_vid.read()
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)
    
    # set video sizes
    size_vid = np.array([frame.shape[1],frame.shape[0]])
    dot_size = 16
    
    #
    fname_out = fname_video.replace('.mp4','_labeled_'+str(start)+"_"+str(end)+'.mp4')
    fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
    video_out = cv2.VideoWriter(fname_out,fourcc, 25, (size_vid[0],size_vid[1]), True)

    #
    dot_size = 5

    # setup cutoff
    pcutoff = 0.5
    font = cv2.FONT_HERSHEY_PLAIN

    #
    for n in trange(start, end, 1):
        ret, frame = original_vid.read()
        if n==start:
            print ("Frame size read: ", frame.shape)

        cv2.putText(frame, str(n)+ "  " +str(round(n/15.,1)),
                    (50, 50),
                    font, 3,
                    (255, 255, 0),
                    5)


        # loop over the featuers
        for f in range(traces.shape[0]):
            if traces[f,n,2]<pcutoff:
                continue

            # check if movement occured
            if movements[f][n]==1:
                dot_size=15
            else:
                dot_size=5

            #
            x = int(traces[f,n,0])
            y = int(traces[f,n,1])
            frame[y-dot_size:y+dot_size,x-dot_size:x+dot_size]= (np.float32(
                matplotlib.colors.to_rgb(clrs[f]))*255.).astype('uint8')
                #matplotlib.colors.to_rgb('white'))*255.).astype('uint8')

        video_out.write(frame)

    video_out.release()
    original_vid.release()
    cv2.destroyAllWindows()

In [41]:
# traces = np.load('/media/cat/4TBSSD/yuki/IA3/tif_files/IA3pm_Feb11_30Hz/IA3_2016-02-11_14-19-04.641DLC_resnet50_yuki_leverJul21shuffle1_100000.npy')
# fname_video = '/media/cat/4TBSSD/yuki/IA3/vids/prestroke/IA3_2016-02-11_14-19-04.641.mp4'
# print (traces.shape)

# data = np.load('/media/cat/4TBSSD/yuki/IA3/tif_files/IA3pm_Feb2_30Hz/IA3pm_Feb2_30Hz_5secNoMove_movements.npz', allow_pickle=True)
# movements = data['feature_movements_times']
# print ("movements: ", movements.shape)
#

traces = np.load('/media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Feb1_30Hz/IJ2_2016-02-01_15-31-33.662DLC_resnet50_yuki_leverJul21shuffle1_278000.npy')
fname_video = '/media/cat/4TBSSD/yuki/IJ2/vids/prestroke/IJ2_2016-02-01_15-31-33.662.mp4'
print (traces.shape)

data = np.load('/media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Feb1_30Hz/IJ2pm_Feb1_30Hz_5secNoMove_movements.npz', allow_pickle=True)
movements = data['feature_movements_times']
print ("movements: ", movements[0].shape)


# 
start = 478*15
end = start+60*15
make_video_dlc(traces,
               movements,
               fname_video,
               start,
               end)

  0%|          | 0/900 [00:00<?, ?it/s]

(7, 19924, 3)
movements:  (7, 19981)
(7, 20782, 3)
movements:  (20782,)
Traces:  (7, 20782, 3)
Frame size read:  (720, 1280, 3)


100%|██████████| 900/900 [00:14<00:00, 61.02it/s]
